# Practical Work

In [21]:
import pandas as pd

## Create the dataset files for RecBole

### Dataset structure
Recbole knowledge aware datasets require three files: .inter, .g, .link files. 

| File | Description |
|------|-------------|
|.inter|User-Item interaction|
|.kg| head, relation, tail|
|.link|item_id to entity_id|

## Create atomic files

 Add type info to kg generated by David

In [22]:
dataset = '20percent_subset'
path = 'data/lfm'
kg_path = '%s/intermediate_kg/%s' % (path, dataset)
prefixed_path = '%s/%s' % (path,dataset)

In [23]:
kg = pd.read_csv('%s_kg.txt' % kg_path, sep='\t', names=['head_id:token', 'relation_id:token', 'tail_id:token'])

In [24]:
kg.head()

,head_id:token,relation_id:token,tail_id:token
0,t36346257,created_by,a1721897
1,t38693544,created_by,a1583455
2,t28859870,created_by,a2754259
3,t28579456,created_by,a3340806
4,t35125510,created_by,a1549805


In [25]:
kg['relation_id:token'].unique()

array(['created_by', 'has_genre', 'lives_in', 'listened_to', 'in_album',
       'has_micro_genre', 'has_gender'], dtype=object)

In [26]:
kg_no_listen_events = kg[kg['relation_id:token'] != 'listened_to']
kg_no_listen_events.to_csv('data/rb_lfm/rb_lfm.kg', sep='\t', index=False)

In [27]:
users = pd.read_csv('%s_users.tsv' % prefixed_path, sep='\t', skiprows=[0], names=['user_id:token', 'country:token', 'age:token', 'gender:token', 'creation_time:token'])
users.to_csv('data/rb_lfm/rb_lfm.user', sep='\t', index=False)

In [28]:
items = pd.read_csv('%s_tracks.tsv' % prefixed_path, sep='\t', skiprows=[0], names=['item_id:token', 'artist:token', 'track:token'])
items.to_csv('data/rb_lfm/rb_lfm.item', sep='\t', index=False)
                    
track_ids = pd.DataFrame(items['item_id:token'])
track_ids['entity_id:token'] = 't' + track_ids['item_id:token'].astype(str)
track_ids.to_csv('data/rb_lfm/rb_lfm.link', sep='\t', index=False)

In [29]:
listening_events = pd.read_csv('%s_listening_events.tsv' % prefixed_path, sep='\t', skiprows=[0], names=['user_id:token', 'item_id:token', 'album_id:token', 'timestamp:token'])
listening_events.to_csv('data/rb_lfm/rb_lfm.inter', sep='\t', index=False)

In [30]:
# free memory
del kg
del kg_no_listen_events
del items
del track_ids
del users

## Demo run model

In [1]:
from recbole.quick_start import run_recbole
run_recbole(model='KGAT', dataset='rb_lfm', config_file_list=['lfm.yaml'])

30 Jan 20:24    INFO  ['/home/jfallmann/miniconda3/envs/pr/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/jfallmann/.local/share/jupyter/runtime/kernel-40903d3c-1567-4f18-bcc8-39e2e53e475b.json']
30 Jan 20:24    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ./data/rb_lfm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics 

KeyboardInterrupt: 

# Custom pipeline

### Load config and create Dataset

In [1]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.knowledge_aware_recommender import KGAT
from recbole.trainer import KGATTrainer
from recbole.utils import init_seed, init_logger

In [2]:

# configurations initialization
config = Config(model='KGAT', dataset='rb_lfm', config_file_list=['lfm.yaml'])

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# write config info into log
logger.info(config)

# dataset creating and filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)


30 Jan 20:26    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ./data/rb_lfm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 2048
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separa

### Train model and evaluate

In [3]:
# model loading and initialization
model = KGAT(config, train_data._dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = KGATTrainer(config, model)

print('Starting to fit model')
# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, saved=True, show_progress=True)

print('Evaluating model')
# model evaluation
test_result = trainer.evaluate(test_data, load_best_model=True)
print(test_result)

/home/jfallmann/miniconda3/envs/pr/lib/python3.9/site-packages/dgl/subgraph.py:288: DGLWarning: Key word argument preserve_nodes is deprecated. Use relabel_nodes instead.
  dgl_warning(
/home/jfallmann/miniconda3/envs/pr/lib/python3.9/site-packages/recbole/model/knowledge_aware_recommender/kgat.py:159: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1).flatten()
/home/jfallmann/miniconda3/envs/pr/lib/python3.9/site-packages/recbole/model/knowledge_aware_recommender/kgat.py:166: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/torch/csrc/utils/tensor_new.cpp:230.)
  indices = torch.LongTensor([final_adj_matrix.row, final_adj_matrix.col])
30 Jan 20:27    INFO  KGAT(
  (user_embedding): Embedding(14683, 16)
  (entity_embedding): Embedding(403239

Starting to fit model


Train     0:   0%|          | 0/1249 [00:00<?, ?it/s]

Train     0:   0%|          | 0/549 [00:00<?, ?it/s]

30 Jan 20:28    INFO  epoch 0 training [time: 75.15s, train_loss1: 512.6705, train_loss2: 168.8829]


...

30 Jan 21:05    INFO  epoch 26 training [time: 60.34s, train_loss1: 65.2837, train_loss2: 3.9077]


Evaluate   :   0%|                                                        | 0/14210 [00:00<?, ?it/s…

30 Jan 21:06    INFO  epoch 26 evaluating [time: 27.66s, valid_score: 0.127900]
30 Jan 21:06    INFO  valid result: 
recall@10 : 0.0301    mrr@10 : 0.1279    ndcg@10 : 0.0561    hit@10 : 0.2737    precision@10 : 0.0448
30 Jan 21:06    INFO  Saving current: saved/KGAT-Jan-30-2023_20-27-14.pth


Train    27:   0%|          | 0/1249 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test_result = trainer.evaluate(test_data, load_best_model=True)
print(test_result)

30 Jan 21:06    INFO  Loading model structure and parameters from saved/KGAT-Jan-30-2023_20-27-14.pth
